In [24]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [25]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)
library(dplyr)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"

GTEx_DATA_DIR <- IN_DIR
GTEx_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GTEx_DATA_DIR,GTEx_DATA_FIL,sep=""))

In [ ]:
%%R

variances <- apply(assay(final_result), 1, var)

zero_variance_rows <- which(variances == 0)

if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data.RData"))
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    final_result <- new_final_result
    
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0     50     50      0      0      0      0
ENSMUSG00000095250.1     49     50      0      0     50      0     49      0
                     [,382] [,383]
ENSMUSG00000079800.2      0      0
ENSMUSG00000095092.1      0      0
ENSMUSG00000079192.2      0      0
ENSMUSG00000079794.2      0      0
ENSMUSG00000094799.1      0      0
ENSMUSG00000095250.1     48      0


In [ ]:
%%R
library(tibble)

keep_samples <- colData(final_result)$dose %in% c(0.00, 3.00)

final_result <- final_result[, keep_samples]

temp_df <- as_tibble(colData(final_result)) %>%
  filter(!grepl("SRP131784", study))

tcdd_data <- temp_df
final_result <- final_result[, colData(final_result)$external_id %in% temp_df$external_id]

print(dim(final_result))
print(assay(final_result))
print(dim(tcdd_data))
print(tcdd_data)

[1] 43252    92
                          [,1]     [,2]     [,3]     [,4]     [,5]     [,6]
ENSMUSG00000079800.2         0        0        0        0        0       50
ENSMUSG00000095092.1         0        0        0        0        0        0
ENSMUSG00000079192.2         0        0        0        0        0        0
ENSMUSG00000079794.2         0        0        0        0        0        0
ENSMUSG00000094799.1         0        0        0        0        0        0
ENSMUSG00000095250.1         0        0        0        0        0        0
ENSMUSG00000095787.1         0        0        0        0        0        0
ENSMUSG00000096100.1         0        0        0        0        0        0
ENSMUSG00000094054.1         0        0        0        0        0        0
ENSMUSG00000095672.1         0        0        0        0        0        0
ENSMUSG00000079190.3         0       50        0        0      150      150
ENSMUSG00000094514.1        49        0        0       96        0      

ENSMUSG00000103067.1         0        0        0        0        0        0
ENSMUSG00000104226.1       100       50       50      100      150        0
ENSMUSG00000103903.1        50        0        0        0        0       50
ENSMUSG00000102647.1         0        0      100        0        0        0
ENSMUSG00000086235.1       598      500      399      800      650      150
ENSMUSG00000102253.1         0        0        0        0        0        0
ENSMUSG00000103884.1         0        0        0        0        0        0
ENSMUSG00000084353.1      4057     5357     4781     2862     2448     3574
ENSMUSG00000103933.1     31020    29109    46442    34434    34721    30695
ENSMUSG00000076135.1         0        8        0        0        0        0
ENSMUSG00000086195.1        36       68       20        0        0       24
ENSMUSG00000104504.1        31        6        0       12       18       26
ENSMUSG00000066693.2      3448     3942     5114     3616     4694     4299
ENSMUSG00000

ENSMUSG00000104165.1         0        0        0        0        0        0
ENSMUSG00000100868.1        50       50        0        0        0        0
ENSMUSG00000089982.7         0        0        0        0        0        0
ENSMUSG00000025920.19     1547     1878     3114     2520     2405     3553
ENSMUSG00000025939.18    10834    10995    12300    10764    10840     9985
ENSMUSG00000102457.1         0        0        0        0        0        0
ENSMUSG00000079658.9     18194    17088    25529    18559    18636    17940
ENSMUSG00000097744.1         0        0        0        0        0        0
ENSMUSG00000100959.1         0        0      100       49        0        0
ENSMUSG00000025940.6     21675    21923    28179    22316    23280    21717
ENSMUSG00000025779.10     1239     1336     2529     2267     2509     2235
ENSMUSG00000100093.1         0        0       50        0        0        0
ENSMUSG00000091020.3      2844     3042     3237     3246     2149     2149
ENSMUSG00000

ENSMUSG00000101301.1         0        0        0        0        0        0
ENSMUSG00000104221.1         0        0        0        0        0        0
ENSMUSG00000102317.1         0        0        0        0        0        0
ENSMUSG00000102659.1         0        0        0        0        0        0
ENSMUSG00000102747.1         0        0        0        0        0        0
ENSMUSG00000102386.1         0        0        0        0        0        0
ENSMUSG00000033569.17        0        0        0        0        0        0
ENSMUSG00000104516.1         0        0        0        0        0        0
ENSMUSG00000099458.1         0        0       45       43       43        0
ENSMUSG00000100310.1         0        0        0        0        0        0
ENSMUSG00000101033.1      1869     2445     5464     3157     2983     3888
ENSMUSG00000101514.1      5324     5359    14206    10611     9326    13754
ENSMUSG00000101372.1       932     1435      945     1782     1288     1033
ENSMUSG00000

ENSMUSG00000102762.1         0        0        0       50        0        0
ENSMUSG00000098680.1         7        2        3        2        2       12
ENSMUSG00000010453.12    33948    32801    50404    30156    29349    38217
ENSMUSG00000037432.15     1590     1622     1246     1109     1351     1308
ENSMUSG00000001143.13     8646     7832    13080     9712     8308     6883
ENSMUSG00000104316.1       100        0       43       49        0        0
ENSMUSG00000106081.1        29       62       42       31       54      119
ENSMUSG00000037408.10     2984     2392     2745     3346     2436     2976
ENSMUSG00000102845.1        77      160      106       79        2      118
ENSMUSG00000001138.13    11274    11022    14452    12200    11106    13414
ENSMUSG00000067653.12     3711     3335     3857     2107     2136     1716
ENSMUSG00000109510.1      3937     4595     3211     2612     2567     2449
ENSMUSG00000079610.9      3640     4078     4563     3317     3249     2190
ENSMUSG00000

...

ENSMUSG00000102664.1        50        0
ENSMUSG00000005886.14    21156    23861
ENSMUSG00000101476.1         0        0
ENSMUSG00000101171.1      1346     1199
ENSMUSG00000025935.10    64108    60872
ENSMUSG00000102982.1      1889     1746
ENSMUSG00000025937.6    106601   109038
ENSMUSG00000067813.3     12975    13310
ENSMUSG00000097711.1       100        0
ENSMUSG00000089358.1         0        0
ENSMUSG00000104209.1         0        0
ENSMUSG00000025932.14        0        0
ENSMUSG00000025930.6       150       50
ENSMUSG00000054493.2         0        0
ENSMUSG00000103492.1         0        0
ENSMUSG00000081201.1      2233     1710
ENSMUSG00000082193.1       889     1587
ENSMUSG00000032769.5         0        0
ENSMUSG00000073737.3         0        0
ENSMUSG00000104149.1        50       99
ENSMUSG00000092083.4         0        0
ENSMUSG00000101652.1       100       49
ENSMUSG00000100814.1         0        0
ENSMUSG00000025925.14     4796     4246
ENSMUSG00000104379.1        50        0


ENSMUSG00000096992.3         0        0
ENSMUSG00000103001.1       879      691
ENSMUSG00000026131.20    49472    48032
ENSMUSG00000104397.1         0        0
ENSMUSG00000102839.1         0        0
ENSMUSG00000101206.1         0        0
ENSMUSG00000101462.2         0        0
ENSMUSG00000092852.1         0        0
ENSMUSG00000102336.1       150      200
ENSMUSG00000042111.9      4890     2700
ENSMUSG00000101711.1       398      692
ENSMUSG00000026127.13    13423    13263
ENSMUSG00000100280.2        50       50
ENSMUSG00000026126.15       50      149
ENSMUSG00000101223.6         0        0
ENSMUSG00000026125.9         0        0
ENSMUSG00000101803.1         0        0
ENSMUSG00000045174.10        0        0
ENSMUSG00000118272.1         0        0
ENSMUSG00000104002.2        50      100
ENSMUSG00000037509.21      261      268
ENSMUSG00000104358.1       298      100
ENSMUSG00000037503.12    20634    20278
ENSMUSG00000026123.11     8161     7611
ENSMUSG00000103716.1         0        0


ENSMUSG00000099381.1         0       50
ENSMUSG00000025993.10    71302    73879
ENSMUSG00000099946.1         0      100
ENSMUSG00000070896.2       195      247
ENSMUSG00000090682.4       698      350
ENSMUSG00000044457.8         0        0
ENSMUSG00000099432.1         0        0
ENSMUSG00000041144.11        0        0
ENSMUSG00000101262.1      4216     3831
ENSMUSG00000101337.6         0        0
ENSMUSG00000100213.1        58      123
ENSMUSG00000103129.1         0        0
ENSMUSG00000025986.6      1298     1098
ENSMUSG00000100160.1       367      384
ENSMUSG00000101447.1         0        0
ENSMUSG00000081185.1      6547     6127
ENSMUSG00000100033.1     11698    11201
ENSMUSG00000094968.1         0        0
ENSMUSG00000101996.1         0        0
ENSMUSG00000096973.1         0       48
ENSMUSG00000064943.1         0        0
ENSMUSG00000101721.1         0        0
ENSMUSG00000097883.1         0        0
ENSMUSG00000099454.1         0        0
ENSMUSG00000051616.14        0        0


ENSMUSG00000081562.1      3903     3773
ENSMUSG00000084400.1         0        0
ENSMUSG00000026014.15    25736    24643
ENSMUSG00000082160.1         0        0
ENSMUSG00000103215.1         0        0
ENSMUSG00000088973.1         0        0
ENSMUSG00000084151.1         0        0
ENSMUSG00000101188.1     58440    58554
ENSMUSG00000026012.2       100      348
ENSMUSG00000104069.1         0        0
ENSMUSG00000086795.1         0        0
ENSMUSG00000083118.1         0        0
ENSMUSG00000085965.1         0        0
ENSMUSG00000083355.1         0        0
ENSMUSG00000102549.1         0        0
ENSMUSG00000081382.1     12625    11168
ENSMUSG00000026011.13        0       50
ENSMUSG00000080800.1         0       50
ENSMUSG00000102561.1         0        0
ENSMUSG00000026009.14      249      248
ENSMUSG00000082804.3         0        0
ENSMUSG00000080785.1         0        0
ENSMUSG00000085248.2         0        0
ENSMUSG00000085447.1         0        0
ENSMUSG00000089546.1         0        0


ENSMUSG00000061815.11      300      482
ENSMUSG00000026180.8        50       50
ENSMUSG00000048480.5         0        0
ENSMUSG00000006304.14    36231    36432
ENSMUSG00000064272.3         0        0
ENSMUSG00000101503.1      3514     4276
ENSMUSG00000006299.13    59574    61975
ENSMUSG00000103013.1       100       50
ENSMUSG00000099931.1       150      200
ENSMUSG00000006301.17    34372    33839
ENSMUSG00000103403.1         0        0
ENSMUSG00000026179.14    23613    26613
ENSMUSG00000073650.8         0       48
ENSMUSG00000026177.11     2066     2810
ENSMUSG00000065468.1        57       50
ENSMUSG00000026176.13    39369    37310
ENSMUSG00000026175.12      148       50
ENSMUSG00000103227.1         0       50
ENSMUSG00000026174.9      9644    10484
ENSMUSG00000033364.13     5173     4249
ENSMUSG00000026173.15      387      337
ENSMUSG00000093155.1         0        0
ENSMUSG00000026135.17     2446     2514
ENSMUSG00000026172.12     6616     5266
ENSMUSG00000026171.12     6130     5914


In [ ]:
%%R
dose_counts <- table(tcdd_data$dose)

print(dose_counts)

dose_counts_df <- as.data.frame(dose_counts)

colnames(dose_counts_df) <- c("Dose", "Count")

write.csv(dose_counts_df, paste0(OUT_DIR, "dose_counts3vs0_time_course.csv"), row.names=FALSE)


 0  3 
68 24 


In [29]:
%%R
combined_gender <- ifelse(tcdd_data$Sex != "", tcdd_data$Sex, tcdd_data$gender)
print(combined_gender)

 [1] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [9] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[17] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[25] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[33] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[41] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[49] "female" "female" "female" "female" "female" "female" "female" "female"
[57] "female" "female" "female" "female" "female" "female" "female" "female"
[65] "female" "female" "female" "female" "female" "female" "female" "female"
[73] "female" "female" "female" "female" "female" "female" "female" "female"
[81] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[89] "male"   "male"   "male"   "male"  


In [30]:
%%R
write.table(tcdd_data$project_id,file=paste(OUT_DIR,"tcdd_project_id3vs0_time_course.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [31]:
%%R
write.table(combined_gender,file=paste(OUT_DIR,"tcdd_gender3vs0_time_course.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [32]:
%%R
write.table(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_id3vs0_time_course.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [33]:
%%R
write.table(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose3vs0_time_course.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [34]:
%%R
saveRDS(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose_detail_vec3vs0_time_course.Rds",sep=""))

In [35]:
%%R
saveRDS(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_detail_vec3vs0_time_course.Rds",sep=""))

In [36]:
%%R
tcdd.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()

In [37]:
%%R
colnames(tcdd.df) <- colData(final_result)$external_id

In [38]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(tcdd.df))
# print(ensembl_wo_ids)

In [39]:
%%R
deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))
print(length(deleted_ensembl_wo_ids))

[1] 12169


In [40]:
%%R
rownames(tcdd.df) <- ensembl_wo_ids
# print(rownames(tcdd.df))

In [41]:
%%R
rownames(deleted_data) <- deleted_ensembl_wo_ids
print(rownames(deleted_data))

    [1] "ENSMUSG00000096776" "ENSMUSG00000096244" "ENSMUSG00000095450"
    [4] "ENSMUSG00000089163" "ENSMUSG00000099278" "ENSMUSG00000094121"
    [7] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000094741"
   [10] "ENSMUSG00000074720" "ENSMUSG00000094337" "ENSMUSG00000095570"
   [13] "ENSMUSG00000098647" "ENSMUSG00000094431" "ENSMUSG00000095247"
   [16] "ENSMUSG00000095585" "ENSMUSG00000096873" "ENSMUSG00000095755"
   [19] "ENSMUSG00000095207" "ENSMUSG00000093828" "ENSMUSG00000095623"
   [22] "ENSMUSG00000095666" "ENSMUSG00000096680" "ENSMUSG00000096236"
   [25] "ENSMUSG00000096756" "ENSMUSG00000094722" "ENSMUSG00000095728"
   [28] "ENSMUSG00000094836" "ENSMUSG00000094383" "ENSMUSG00000094474"
   [31] "ENSMUSG00000096550" "ENSMUSG00000094172" "ENSMUSG00000091585"
   [34] "ENSMUSG00000095763" "ENSMUSG00000095523" "ENSMUSG00000095475"
   [37] "ENSMUSG00000095019" "ENSMUSG00000102693" "ENSMUSG00000064842"
   [40] "ENSMUSG00000102851" "ENSMUSG00000103377" "ENSMUSG00000102348"
   [43

  [346] "ENSMUSG00000104104" "ENSMUSG00000087883" "ENSMUSG00000099892"
  [349] "ENSMUSG00000100977" "ENSMUSG00000077039" "ENSMUSG00000100684"
  [352] "ENSMUSG00000099770" "ENSMUSG00000088510" "ENSMUSG00000101890"
  [355] "ENSMUSG00000087908" "ENSMUSG00000088594" "ENSMUSG00000100955"
  [358] "ENSMUSG00000099469" "ENSMUSG00000101297" "ENSMUSG00000100059"
  [361] "ENSMUSG00000088161" "ENSMUSG00000100935" "ENSMUSG00000103991"
  [364] "ENSMUSG00000084704" "ENSMUSG00000103060" "ENSMUSG00000089866"
  [367] "ENSMUSG00000089206" "ENSMUSG00000084228" "ENSMUSG00000044594"
  [370] "ENSMUSG00000058017" "ENSMUSG00000080729" "ENSMUSG00000073602"
  [373] "ENSMUSG00000073601" "ENSMUSG00000087210" "ENSMUSG00000089278"
  [376] "ENSMUSG00000077237" "ENSMUSG00000099484" "ENSMUSG00000104478"
  [379] "ENSMUSG00000101231" "ENSMUSG00000100884" "ENSMUSG00000089037"
  [382] "ENSMUSG00000084741" "ENSMUSG00000100290" "ENSMUSG00000100758"
  [385] "ENSMUSG00000102728" "ENSMUSG00000088691" "ENSMUSG00000099884"
  [388

  [691] "ENSMUSG00000077749" "ENSMUSG00000111753" "ENSMUSG00000088423"
  [694] "ENSMUSG00000044499" "ENSMUSG00000103299" "ENSMUSG00000111549"
  [697] "ENSMUSG00000096080" "ENSMUSG00000077576" "ENSMUSG00000111234"
  [700] "ENSMUSG00000088925" "ENSMUSG00000111048" "ENSMUSG00000111208"
  [703] "ENSMUSG00000110782" "ENSMUSG00000094521" "ENSMUSG00000094311"
  [706] "ENSMUSG00000111183" "ENSMUSG00000019834" "ENSMUSG00000111098"
  [709] "ENSMUSG00000110889" "ENSMUSG00000046922" "ENSMUSG00000111576"
  [712] "ENSMUSG00000094663" "ENSMUSG00000111315" "ENSMUSG00000093298"
  [715] "ENSMUSG00000087891" "ENSMUSG00000099893" "ENSMUSG00000098642"
  [718] "ENSMUSG00000076282" "ENSMUSG00000110891" "ENSMUSG00000077082"
  [721] "ENSMUSG00000098418" "ENSMUSG00000077591" "ENSMUSG00000098993"
  [724] "ENSMUSG00000098409" "ENSMUSG00000111182" "ENSMUSG00000094681"
  [727] "ENSMUSG00000065718" "ENSMUSG00000111499" "ENSMUSG00000111419"
  [730] "ENSMUSG00000110843" "ENSMUSG00000111719" "ENSMUSG00000112062"
  [733

 [7861] "ENSMUSG00000109318" "ENSMUSG00000096859" "ENSMUSG00000109210"
 [7864] "ENSMUSG00000062598" "ENSMUSG00000109065" "ENSMUSG00000092456"
 [7867] "ENSMUSG00000118562" "ENSMUSG00000108430" "ENSMUSG00000108404"
 [7870] "ENSMUSG00000108302" "ENSMUSG00000108675" "ENSMUSG00000108809"
 [7873] "ENSMUSG00000094866" "ENSMUSG00000054102" "ENSMUSG00000064793"
 [7876] "ENSMUSG00000118574" "ENSMUSG00000060747" "ENSMUSG00000089219"
 [7879] "ENSMUSG00000065385" "ENSMUSG00000065659" "ENSMUSG00000109467"

 [8374] "ENSMUSG00000108770" "ENSMUSG00000098910" "ENSMUSG00000108384"
 [8377] "ENSMUSG00000108721" "ENSMUSG00000093212" "ENSMUSG00000086232"
 [8380] "ENSMUSG00000065800" "ENSMUSG00000087834" "ENSMUSG00000108924"
 [8383] "ENSMUSG00000109298" "ENSMUSG00000094854" "ENSMUSG00000065940"
 [8386] "ENSMUSG00000109400" "ENSMUSG00000110028" "ENSMUSG00000109153"
 [8389] "ENSMUSG00000108885" "ENSMUSG00000109412" "ENSMUSG00000108821"
 [8392] "ENSMUSG00000077166" "ENSMUSG00000109280" "ENSMUSG00000109018"
 [839

 [8698] "ENSMUSG00000108837" "ENSMUSG00000108998" "ENSMUSG00000058244"
 [8701] "ENSMUSG00000061000" "ENSMUSG00000063764" "ENSMUSG00000096209"
 [8704] "ENSMUSG00000109070" "ENSMUSG00000056946" "ENSMUSG00000051200"
 [8707] "ENSMUSG00000109207" "ENSMUSG00000066241" "ENSMUSG00000109406"
 [8710] "ENSMUSG00000062434" "ENSMUSG00000046431" "ENSMUSG00000066239"
 [8713] "ENSMUSG00000109001" "ENSMUSG00000109749" "ENSMUSG00000077902"
 [8716] "ENSMUSG00000065751" "ENSMUSG00000093002" "ENSMUSG00000110207"
 [8719] "ENSMUSG00000088671" "ENSMUSG00000110009" "ENSMUSG00000059077"
 [8722] "ENSMUSG00000109779" "ENSMUSG00000109968" "ENSMUSG00000089467"
 [8725] "ENSMUSG00000073862" "ENSMUSG00000101755" "ENSMUSG00000089348"
 [8728] "ENSMUSG00000108419" "ENSMUSG00000108566" "ENSMUSG00000084492"
 [8731] "ENSMUSG00000088234" "ENSMUSG00000108303" "ENSMUSG00000088116"
 [8734] "ENSMUSG00000118412" "ENSMUSG00000077653" "ENSMUSG00000109414"
 [8737] "ENSMUSG00000109578" "ENSMUSG00000088935" "ENSMUSG00000108697"
 [8740

...

[10423] "ENSMUSG00000100133" "ENSMUSG00000088313" "ENSMUSG00000083321"
[10426] "ENSMUSG00000077435" "ENSMUSG00000087716" "ENSMUSG00000088539"
[10429] "ENSMUSG00000082590" "ENSMUSG00000084054" "ENSMUSG00000086452"
[10432] "ENSMUSG00000083490" "ENSMUSG00000094864" "ENSMUSG00000091821"
[10435] "ENSMUSG00000095034" "ENSMUSG00000094288" "ENSMUSG00000094999"
[10438] "ENSMUSG00000085536" "ENSMUSG00000095108" "ENSMUSG00000095420"
[10441] "ENSMUSG00000094238" "ENSMUSG00000118451" "ENSMUSG00000096318"
[10444] "ENSMUSG00000093990" "ENSMUSG00000118453" "ENSMUSG00000094675"
[10447] "ENSMUSG00000096135" "ENSMUSG00000118442" "ENSMUSG00000095283"
[10450] "ENSMUSG00000095017" "ENSMUSG00000118468" "ENSMUSG00000118470"
[10453] "ENSMUSG00000118475" "ENSMUSG00000118432" "ENSMUSG00000094540"
[10456] "ENSMUSG00000118484" "ENSMUSG00000095008" "ENSMUSG00000095499"
[10459] "ENSMUSG00000094342" "ENSMUSG00000118464" "ENSMUSG00000096897"
[10462] "ENSMUSG00000118418" "ENSMUSG00000095563" "ENSMUSG00000081736"
[10465

[10768] "ENSMUSG00000095148" "ENSMUSG00000101707" "ENSMUSG00000100289"
[10771] "ENSMUSG00000101243" "ENSMUSG00000100479" "ENSMUSG00000099352"
[10774] "ENSMUSG00000101055" "ENSMUSG00000099505" "ENSMUSG00000100161"
[10777] "ENSMUSG00000102039" "ENSMUSG00000101913" "ENSMUSG00000094484"
[10780] "ENSMUSG00000101479" "ENSMUSG00000100939" "ENSMUSG00000100262"
[10783] "ENSMUSG00000100453" "ENSMUSG00000100858" "ENSMUSG00000099965"
[10786] "ENSMUSG00000100948" "ENSMUSG00000118447" "ENSMUSG00000100961"
[10789] "ENSMUSG00000100806" "ENSMUSG00000101521" "ENSMUSG00000101181"
[10792] "ENSMUSG00000100645" "ENSMUSG00000100124" "ENSMUSG00000101956"
[10795] "ENSMUSG00000101351" "ENSMUSG00000100412" "ENSMUSG00000102026"
[10798] "ENSMUSG00000099800" "ENSMUSG00000099861" "ENSMUSG00000095324"
[10801] "ENSMUSG00000101616" "ENSMUSG00000100312" "ENSMUSG00000100258"
[10804] "ENSMUSG00000101229" "ENSMUSG00000101456" "ENSMUSG00000103618"
[10807] "ENSMUSG00000100421" "ENSMUSG00000101928" "ENSMUSG00000100239"
[10810

[11113] "ENSMUSG00000101839" "ENSMUSG00000102010" "ENSMUSG00000101149"
[11116] "ENSMUSG00000101743" "ENSMUSG00000101311" "ENSMUSG00000100803"
[11119] "ENSMUSG00000100873" "ENSMUSG00000101692" "ENSMUSG00000100464"
[11122] "ENSMUSG00000102015" "ENSMUSG00000101198" "ENSMUSG00000099955"
[11125] "ENSMUSG00000099873" "ENSMUSG00000099541" "ENSMUSG00000101920"
[11128] "ENSMUSG00000099430" "ENSMUSG00000100099" "ENSMUSG00000100495"
[11131] "ENSMUSG00000100413" "ENSMUSG00000100570" "ENSMUSG00000101863"
[11134] "ENSMUSG00000099727" "ENSMUSG00000099791" "ENSMUSG00000101473"
[11137] "ENSMUSG00000101753" "ENSMUSG00000101484" "ENSMUSG00000100587"
[11140] "ENSMUSG00000099828" "ENSMUSG00000101158" "ENSMUSG00000099537"
[11143] "ENSMUSG00000101669" "ENSMUSG00000099862" "ENSMUSG00000101908"
[11146] "ENSMUSG00000100473" "ENSMUSG00000101192" "ENSMUSG00000101607"
[11149] "ENSMUSG00000101418" "ENSMUSG00000099431" "ENSMUSG00000101837"
[11152] "ENSMUSG00000100233" "ENSMUSG00000100859" "ENSMUSG00000099653"
[11155

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[11860] "ENSMUSG00000099371" "ENSMUSG00000102129" "ENSMUSG00000100359"
[11863] "ENSMUSG00000099495" "ENSMUSG00000101747" "ENSMUSG00000100379"
[11866] "ENSMUSG00000101715" "ENSMUSG00000101425" "ENSMUSG00000100698"
[11869] "ENSMUSG00000099547" "ENSMUSG00000099782" "ENSMUSG00000101038"
[11872] "ENSMUSG00000100604" "ENSMUSG00000099554" "ENSMUSG00000101757"
[11875] "ENSMUSG00000100179" "ENSMUSG00000099685" "ENSMUSG00000100251"
[11878] "ENSMUSG00000101774" "ENSMUSG00000099883" "ENSMUSG00000100363"
[11881] "ENSMUSG00000100276" "ENSMUSG00000101318" "ENSMUSG00000100805"
[11884] "ENSMUSG00000101528" "ENSMUSG00000099661" "ENSMUSG00000094507"
[11887] "ENSMUSG00000100115" "ENSMUSG00000093946" "ENSMUSG00000101881"
[11890] "ENSMUSG00000101218" "ENSMUSG00000094654" "ENSMUSG00000101128"
[11893] "ENSMUSG00000101788" "ENSMUSG00000102087" "ENSMUSG00000101428"
[11896] "ENSMUSG00000100140" "ENSMUSG00000101146" "ENSMUSG00000099499"
[11899] "ENSMUSG00000101213" "ENSMUSG00000099896" "ENSMUSG00000101980"
[11902

In [42]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)

   [1] "ENSMUSG00000000001" "ENSMUSG00000000028" "ENSMUSG00000000049"
   [4] "ENSMUSG00000000058" "ENSMUSG00000000085" "ENSMUSG00000000088"
   [7] "ENSMUSG00000000120" "ENSMUSG00000000126" "ENSMUSG00000000127"
  [10] "ENSMUSG00000000142" "ENSMUSG00000000149" "ENSMUSG00000000154"
  [13] "ENSMUSG00000000168" "ENSMUSG00000000171" "ENSMUSG00000000182"
  [16] "ENSMUSG00000000183" "ENSMUSG00000000184" "ENSMUSG00000000197"
  [19] "ENSMUSG00000000214" "ENSMUSG00000000215" "ENSMUSG00000000216"
  [22] "ENSMUSG00000000223" "ENSMUSG00000000247" "ENSMUSG00000000253"
  [25] "ENSMUSG00000000263" "ENSMUSG00000000275" "ENSMUSG00000000276"
  [28] "ENSMUSG00000000290" "ENSMUSG00000000296" "ENSMUSG00000000301"
  [31] "ENSMUSG00000000303" "ENSMUSG00000000305" "ENSMUSG00000000308"
  [34] "ENSMUSG00000000320" "ENSMUSG00000000326" "ENSMUSG00000000340"
  [37] "ENSMUSG00000000374" "ENSMUSG00000000394" "ENSMUSG00000000399"
  [40] "ENSMUSG00000000409" "ENSMUSG00000000420" "ENSMUSG00000000435"
  [43] "ENSMUSG00000

 [352] "ENSMUSG00000003541" "ENSMUSG00000003546" "ENSMUSG00000003549"
 [355] "ENSMUSG00000003555" "ENSMUSG00000003559" "ENSMUSG00000003573"
 [358] "ENSMUSG00000003604" "ENSMUSG00000003617" "ENSMUSG00000003623"
 [361] "ENSMUSG00000003644" "ENSMUSG00000003660" "ENSMUSG00000003665"
 [364] "ENSMUSG00000003731" "ENSMUSG00000003746" "ENSMUSG00000003752"
 [367] "ENSMUSG00000003779" "ENSMUSG00000003809" "ENSMUSG00000003812"
 [370] "ENSMUSG00000003813" "ENSMUSG00000003814" "ENSMUSG00000003824"
 [373] "ENSMUSG00000003848" "ENSMUSG00000003849" "ENSMUSG00000003863"
 [376] "ENSMUSG00000003865" "ENSMUSG00000003872" "ENSMUSG00000003873"
 [379] "ENSMUSG00000003882" "ENSMUSG00000003923" "ENSMUSG00000003934"
 [382] "ENSMUSG00000003970" "ENSMUSG00000003974" "ENSMUSG00000004018"
 [385] "ENSMUSG00000004032" "ENSMUSG00000004035" "ENSMUSG00000004038"
 [388] "ENSMUSG00000004040" "ENSMUSG00000004043" "ENSMUSG00000004044"
 [391] "ENSMUSG00000004054" "ENSMUSG00000004056" "ENSMUSG00000004070"
 [394] "ENSMUSG00000

 [703] "ENSMUSG00000008373" "ENSMUSG00000008435" "ENSMUSG00000008438"
 [706] "ENSMUSG00000008461" "ENSMUSG00000008475" "ENSMUSG00000008496"
 [709] "ENSMUSG00000008540" "ENSMUSG00000008590" "ENSMUSG00000008601"
 [712] "ENSMUSG00000008658" "ENSMUSG00000008668" "ENSMUSG00000008682"
 [715] "ENSMUSG00000008683" "ENSMUSG00000008690" "ENSMUSG00000008730"
 [718] "ENSMUSG00000008763" "ENSMUSG00000008845" "ENSMUSG00000008855"
 [721] "ENSMUSG00000008859" "ENSMUSG00000008892" "ENSMUSG00000008932"
 [724] "ENSMUSG00000008976" "ENSMUSG00000008999" "ENSMUSG00000009013"
 [727] "ENSMUSG00000009030" "ENSMUSG00000009073" "ENSMUSG00000009076"
 [730] "ENSMUSG00000009090" "ENSMUSG00000009092" "ENSMUSG00000009108"
 [733] "ENSMUSG00000009246" "ENSMUSG00000009281" "ENSMUSG00000009292"
 [736] "ENSMUSG00000009293" "ENSMUSG00000009350" "ENSMUSG00000009356"
 [739] "ENSMUSG00000009376" "ENSMUSG00000009394" "ENSMUSG00000009406"
 [742] "ENSMUSG00000009470" "ENSMUSG00000009487" "ENSMUSG00000009535"
 [745] "ENSMUSG00000

[1054] "ENSMUSG00000017718" "ENSMUSG00000017721" "ENSMUSG00000017723"
[1057] "ENSMUSG00000017724" "ENSMUSG00000017733" "ENSMUSG00000017737"
[1060] "ENSMUSG00000017740" "ENSMUSG00000017747" "ENSMUSG00000017754"
[1063] "ENSMUSG00000017756" "ENSMUSG00000017760" "ENSMUSG00000017765"
[1066] "ENSMUSG00000017774" "ENSMUSG00000017776" "ENSMUSG00000017778"
[1069] "ENSMUSG00000017781" "ENSMUSG00000017831" "ENSMUSG00000017837"
[1072] "ENSMUSG00000017843" "ENSMUSG00000017858" "ENSMUSG00000017861"
[1075] "ENSMUSG00000017868" "ENSMUSG00000017897" "ENSMUSG00000017929"
[1078] "ENSMUSG00000017950" "ENSMUSG00000017969" "ENSMUSG00000018001"
[1081] "ENSMUSG00000018008" "ENSMUSG00000018012" "ENSMUSG00000018040"
[1084] "ENSMUSG00000018042" "ENSMUSG00000018068" "ENSMUSG00000018102"
[1087] "ENSMUSG00000018126" "ENSMUSG00000018143" "ENSMUSG00000018160"
[1090] "ENSMUSG00000018166" "ENSMUSG00000018167" "ENSMUSG00000018169"
[1093] "ENSMUSG00000018189" "ENSMUSG00000018209" "ENSMUSG00000018263"
[1096] "ENSMUSG00000

[1405] "ENSMUSG00000020308" "ENSMUSG00000020309" "ENSMUSG00000020310"
[1408] "ENSMUSG00000020311" "ENSMUSG00000020312" "ENSMUSG00000020315"
[1411] "ENSMUSG00000020321" "ENSMUSG00000020325" "ENSMUSG00000020326"
[1414] "ENSMUSG00000020327" "ENSMUSG00000020329" "ENSMUSG00000020330"
[1417] "ENSMUSG00000020331" "ENSMUSG00000020333" "ENSMUSG00000020334"
[1420] "ENSMUSG00000020335" "ENSMUSG00000020340" "ENSMUSG00000020346"
[1423] "ENSMUSG00000020349" "ENSMUSG00000020357" "ENSMUSG00000020359"
[1426] "ENSMUSG00000020361" "ENSMUSG00000020362" "ENSMUSG00000020363"
[1429] "ENSMUSG00000020364" "ENSMUSG00000020366" "ENSMUSG00000020368"
[1432] "ENSMUSG00000020372" "ENSMUSG00000020375" "ENSMUSG00000020376"
[1435] "ENSMUSG00000020377" "ENSMUSG00000020380" "ENSMUSG00000020383"
[1438] "ENSMUSG00000020386" "ENSMUSG00000020387" "ENSMUSG00000020390"
[1441] "ENSMUSG00000020393" "ENSMUSG00000020395" "ENSMUSG00000020397"
[1444] "ENSMUSG00000020399" "ENSMUSG00000020400" "ENSMUSG00000020402"
[1447] "ENSMUSG00000

...

[5464] "ENSMUSG00000034321" "ENSMUSG00000034329" "ENSMUSG00000034330"
[5467] "ENSMUSG00000034342" "ENSMUSG00000034343" "ENSMUSG00000034345"
[5470] "ENSMUSG00000034349" "ENSMUSG00000034353" "ENSMUSG00000034354"
[5473] "ENSMUSG00000034371" "ENSMUSG00000034377" "ENSMUSG00000034391"
[5476] "ENSMUSG00000034394" "ENSMUSG00000034402" "ENSMUSG00000034403"
[5479] "ENSMUSG00000034412" "ENSMUSG00000034422" "ENSMUSG00000034424"
[5482] "ENSMUSG00000034432" "ENSMUSG00000034452" "ENSMUSG00000034453"
[5485] "ENSMUSG00000034456" "ENSMUSG00000034457" "ENSMUSG00000034462"
[5488] "ENSMUSG00000034467" "ENSMUSG00000034473" "ENSMUSG00000034480"
[5491] "ENSMUSG00000034484" "ENSMUSG00000034486" "ENSMUSG00000034520"
[5494] "ENSMUSG00000034525" "ENSMUSG00000034528" "ENSMUSG00000034538"
[5497] "ENSMUSG00000034543" "ENSMUSG00000034557" "ENSMUSG00000034566"
[5500] "ENSMUSG00000034570" "ENSMUSG00000034573" "ENSMUSG00000034574"
[5503] "ENSMUSG00000034579" "ENSMUSG00000034591" "ENSMUSG00000034593"
[5506] "ENSMUSG00000

[5815] "ENSMUSG00000036672" "ENSMUSG00000036678" "ENSMUSG00000036693"
[5818] "ENSMUSG00000036698" "ENSMUSG00000036707" "ENSMUSG00000036712"
[5821] "ENSMUSG00000036733" "ENSMUSG00000036743" "ENSMUSG00000036745"
[5824] "ENSMUSG00000036751" "ENSMUSG00000036752" "ENSMUSG00000036760"
[5827] "ENSMUSG00000036766" "ENSMUSG00000036768" "ENSMUSG00000036775"
[5830] "ENSMUSG00000036777" "ENSMUSG00000036781" "ENSMUSG00000036782"
[5833] "ENSMUSG00000036790" "ENSMUSG00000036792" "ENSMUSG00000036805"
[5836] "ENSMUSG00000036810" "ENSMUSG00000036813" "ENSMUSG00000036814"
[5839] "ENSMUSG00000036817" "ENSMUSG00000036819" "ENSMUSG00000036820"
[5842] "ENSMUSG00000036822" "ENSMUSG00000036832" "ENSMUSG00000036833"
[5845] "ENSMUSG00000036834" "ENSMUSG00000036835" "ENSMUSG00000036840"
[5848] "ENSMUSG00000036845" "ENSMUSG00000036850" "ENSMUSG00000036853"
[5851] "ENSMUSG00000036855" "ENSMUSG00000036856" "ENSMUSG00000036858"
[5854] "ENSMUSG00000036860" "ENSMUSG00000036863" "ENSMUSG00000036867"
[5857] "ENSMUSG00000

[6166] "ENSMUSG00000038600" "ENSMUSG00000038607" "ENSMUSG00000038608"
[6169] "ENSMUSG00000038622" "ENSMUSG00000038628" "ENSMUSG00000038630"
[6172] "ENSMUSG00000038633" "ENSMUSG00000038641" "ENSMUSG00000038642"
[6175] "ENSMUSG00000038644" "ENSMUSG00000038648" "ENSMUSG00000038656"
[6178] "ENSMUSG00000038658" "ENSMUSG00000038664" "ENSMUSG00000038665"
[6181] "ENSMUSG00000038668" "ENSMUSG00000038670" "ENSMUSG00000038671"
[6184] "ENSMUSG00000038676" "ENSMUSG00000038677" "ENSMUSG00000038685"
[6187] "ENSMUSG00000038690" "ENSMUSG00000038692" "ENSMUSG00000038695"
[6190] "ENSMUSG00000038696" "ENSMUSG00000038702" "ENSMUSG00000038708"
[6193] "ENSMUSG00000038717" "ENSMUSG00000038722" "ENSMUSG00000038732"
[6196] "ENSMUSG00000038738" "ENSMUSG00000038740" "ENSMUSG00000038751"
[6199] "ENSMUSG00000038754" "ENSMUSG00000038756" "ENSMUSG00000038759"
[6202] "ENSMUSG00000038762" "ENSMUSG00000038764" "ENSMUSG00000038773"
[6205] "ENSMUSG00000038776" "ENSMUSG00000038780" "ENSMUSG00000038781"
[6208] "ENSMUSG00000

[6517] "ENSMUSG00000040675" "ENSMUSG00000040680" "ENSMUSG00000040681"
[6520] "ENSMUSG00000040687" "ENSMUSG00000040688" "ENSMUSG00000040690"
[6523] "ENSMUSG00000040693" "ENSMUSG00000040694" "ENSMUSG00000040701"
[6526] "ENSMUSG00000040703" "ENSMUSG00000040706" "ENSMUSG00000040710"
[6529] "ENSMUSG00000040713" "ENSMUSG00000040714" "ENSMUSG00000040717"
[6532] "ENSMUSG00000040724" "ENSMUSG00000040731" "ENSMUSG00000040734"
[6535] "ENSMUSG00000040738" "ENSMUSG00000040747" "ENSMUSG00000040751"
[6538] "ENSMUSG00000040752" "ENSMUSG00000040760" "ENSMUSG00000040761"
[6541] "ENSMUSG00000040767" "ENSMUSG00000040774" "ENSMUSG00000040782"
[6544] "ENSMUSG00000040797" "ENSMUSG00000040812" "ENSMUSG00000040813"
[6547] "ENSMUSG00000040818" "ENSMUSG00000040820" "ENSMUSG00000040824"
[6550] "ENSMUSG00000040828" "ENSMUSG00000040836" "ENSMUSG00000040850"
[6553] "ENSMUSG00000040855" "ENSMUSG00000040857" "ENSMUSG00000040865"
[6556] "ENSMUSG00000040875" "ENSMUSG00000040891" "ENSMUSG00000040896"
[6559] "ENSMUSG00000

[6868] "ENSMUSG00000043962" "ENSMUSG00000043969" "ENSMUSG00000043972"
[6871] "ENSMUSG00000043987" "ENSMUSG00000043998" "ENSMUSG00000044005"
[6874] "ENSMUSG00000044014" "ENSMUSG00000044018" "ENSMUSG00000044034"
[6877] "ENSMUSG00000044037" "ENSMUSG00000044041" "ENSMUSG00000044052"
[6880] "ENSMUSG00000044080" "ENSMUSG00000044103" "ENSMUSG00000044147"
[6883] "ENSMUSG00000044155" "ENSMUSG00000044162" "ENSMUSG00000044164"
[6886] "ENSMUSG00000044167" "ENSMUSG00000044199" "ENSMUSG00000044201"
[6889] "ENSMUSG00000044216" "ENSMUSG00000044217" "ENSMUSG00000044244"
[6892] "ENSMUSG00000044252" "ENSMUSG00000044254" "ENSMUSG00000044287"
[6895] "ENSMUSG00000044288" "ENSMUSG00000044292" "ENSMUSG00000044294"
[6898] "ENSMUSG00000044309" "ENSMUSG00000044312" "ENSMUSG00000044317"
[6901] "ENSMUSG00000044322" "ENSMUSG00000044337" "ENSMUSG00000044338"
[6904] "ENSMUSG00000044340" "ENSMUSG00000044359" "ENSMUSG00000044393"
[6907] "ENSMUSG00000044408" "ENSMUSG00000044430" "ENSMUSG00000044442"
[6910] "ENSMUSG00000

In [43]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids3vs0.Rds",sep=""))

In [21]:
%%R
tcdd.df <- tcdd.df[reactome_ensembl_ids,]
print(tcdd.df)

                   SRR7817611 SRR7817612 SRR7817614 SRR7817615 SRR7817616
ENSMUSG00000000001      54414      55963      71636      56315      55499
ENSMUSG00000000028        483        371        785        445        508
ENSMUSG00000000049     867859     869565    1380723    1012617    1016838
ENSMUSG00000000058       1200       1446       1997       1048       1967
ENSMUSG00000000085       5970       5071       7537       5648       4328
ENSMUSG00000000088     102144     104159     135652      84658      82419
ENSMUSG00000000120       3593       3842       5318       2860       2708
ENSMUSG00000000126        299        599        300        649        500
ENSMUSG00000000127       3431       2944       5020       3461       4270
ENSMUSG00000000142       3978       3978       5202       6884       5789
ENSMUSG00000000149      63577      67422      87839      70522      62028
ENSMUSG00000000154      46678      50051      69606      53885      54402
ENSMUSG00000000168      30504      315

ENSMUSG00000002249       6461       6663       9627       7581       6350
ENSMUSG00000002250      27687      30336      30212      33623      35205
ENSMUSG00000002257        200        146        230        350        300
ENSMUSG00000002279      16221      20624      27187      22749      21433
ENSMUSG00000002289      12629      11667      31828      11101      12772
ENSMUSG00000002297       1260       1519       1381        945        788
ENSMUSG00000002307       7276       7464      10207       8225       8229
ENSMUSG00000002308       2714       2680       2652       3314       2534
ENSMUSG00000002324       1077        788       2328        448        648
ENSMUSG00000002325      20146      21517      27420      24704      26582
ENSMUSG00000002326       8550       8826      10024       7821       8404
ENSMUSG00000002346      64896      67271      82118      63564      64902
ENSMUSG00000002365      20649      21494      33607      24690      25591
ENSMUSG00000002379      27330      247

ENSMUSG00000004654          0          0          0          0          0
ENSMUSG00000004655      41227      41735      48111      41064      41048
ENSMUSG00000004665       3536       2943       3696       3349       4333
ENSMUSG00000004667      16817      18820      20609      14270      15317
ENSMUSG00000004668          0         50          0          0          0
ENSMUSG00000004677      20901      21276      22605      18446      17750
ENSMUSG00000004709         50        200        100         50        240
ENSMUSG00000004730      11537      12030       5172       8580       8435
ENSMUSG00000004768       5440       4544       7419       5263       5385
ENSMUSG00000004771      31430      34095      43689      31829      30566
ENSMUSG00000004788      11441      12893      14088      10703      10919
ENSMUSG00000004789     124044     127284     159502     122528     127971
ENSMUSG00000004791         50         50          0          0          0
ENSMUSG00000004815      23720      232

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



       1471       1399       2126       1735       1804
ENSMUSG00000007097        349        400        415        200        350
ENSMUSG00000007107          0          0          0         50         38
ENSMUSG00000007122        206        400          0         41         50
ENSMUSG00000007207        519        700        598        250        460
ENSMUSG00000007216       1845       1050       1348       2844       2500
ENSMUSG00000007279          0          0          0          0          0
ENSMUSG00000007338      17345      18131      26285      18831      17980
ENSMUSG00000007379        450        150        999        548        644
ENSMUSG00000007411      25160      28005      33276      26337      26808
ENSMUSG00000007458      51932      56768      80903      55864      60142
ENSMUSG00000007476      15694      17009      16313      13786      11602
ENSMUSG00000007480          0          0          0          0          0
ENSMUSG00000007564      32018      32807      43889     

...

ENSMUSG00000005628        209        300        348         14        250
ENSMUSG00000005667       1448        880       1679       2206        612
ENSMUSG00000005672       2373       2083       2667       2981       3639
ENSMUSG00000005677     141531     140441     119247     124749     154860
ENSMUSG00000005681    3448600    3423247    2905195    2917430    4922987
ENSMUSG00000005682      21374      21413      22650      24363      32367
ENSMUSG00000005683      77711      79613      79942      81474     120821
ENSMUSG00000005686        608        700        150        414       1094
ENSMUSG00000005687      12911      11516      14910      16623      21134
ENSMUSG00000005698      16025      15101      16967      17046      24682
ENSMUSG00000005699       2248       2118       2649       1999       2537
ENSMUSG00000005705         44         63        163        182         84
ENSMUSG00000005716          0          0          0          0          0
ENSMUSG00000005763       1833       12

ENSMUSG00000009905       9320       9194      13306      12775      14994
ENSMUSG00000009907      20952      20905      26603      24949      36899
ENSMUSG00000009927      42453      43427      23529      27106      66756
ENSMUSG00000009941          0          0          0          0          0
ENSMUSG00000009995      17217      19251      21798      19836      29341
ENSMUSG00000010021        187         95        150        150        550
ENSMUSG00000010025     164773     169156     210468     220809     259465
ENSMUSG00000010045      11767      12358      12916      12837      16271
ENSMUSG00000010047      24675      25673      31088      29551      36792
ENSMUSG00000010051      44989      50410      57796      60149      58157
ENSMUSG00000010057       7999       9318       8441       9426      12635
ENSMUSG00000010064     543109     547798     569226     598394     858754
ENSMUSG00000010066          0         48        100         50          0
ENSMUSG00000010095      22907      240

ENSMUSG00000016255         49          0          0        100        250
ENSMUSG00000016256     122116     123295     107739     109027     148240
ENSMUSG00000016283          0         50          0          0          0
ENSMUSG00000016308      10915      13039      12985      13878      16789
ENSMUSG00000016319     208833     209905     233584     242007     284945
ENSMUSG00000016327          0          0         50          0          0
ENSMUSG00000016356       1879       2612       1295       1786       1351
ENSMUSG00000016409       4187       3515       3961       3481       6134
ENSMUSG00000016427      24299      22382      12500      12734      30940
ENSMUSG00000016477       5366       4616       6017       5423       3247
ENSMUSG00000016487      12613      12354      13787      16468      19485
ENSMUSG00000016493       3326       3143       3697       4292       7134
ENSMUSG00000016494        448       1050       1049        999        931
ENSMUSG00000016496        998       11

ENSMUSG00000001127      39117      31137      33777      23590      23523
ENSMUSG00000001128       3868       4896       5027       5290       4493
ENSMUSG00000001131          0         50        132         44        241
ENSMUSG00000001143       5277       7825       8757       4212       5418
ENSMUSG00000001155     301092     261021     257541     147943     142374
ENSMUSG00000001158       5688       4232       5411       6212       6900
ENSMUSG00000001173       3120       1836       2338       3280       3667
ENSMUSG00000001175     110009     108895     110071      73015      75055
ENSMUSG00000001211      99203      91413      92216      50865      49499
ENSMUSG00000001225          0          0          0          0          0
ENSMUSG00000001228        150         99         50        295        350
ENSMUSG00000001240      40692      35084      33257      21944      23065
ENSMUSG00000001247      81474     117467     116776      47585      49157
ENSMUSG00000001249     250124     1855

ENSMUSG00000003418        100          0          0         50          0
ENSMUSG00000003421      10286       7002       7869       6243       4757
ENSMUSG00000003429      27441      26574      28887      34808      32980
ENSMUSG00000003435      29681      37506      35125      17503      18267
ENSMUSG00000003436          0          0          0          0          0
ENSMUSG00000003437      18304      15253      12398      11170      11403
ENSMUSG00000003444       2704       2611       1925       1282        554
ENSMUSG00000003452         49          0         50        141        150
ENSMUSG00000003458      22307      18210      18645      16214      15625
ENSMUSG00000003464      72315      49784      48589      44918      43208
ENSMUSG00000003476         48        250        100         50          0
ENSMUSG00000003477     784262     638003     650291     427986     416743
ENSMUSG00000003484        114        132         92        142        150
ENSMUSG00000003500        649       12

In [22]:
%%R
saveRDS(tcdd.df,file=paste(OUT_DIR,"tcdd_df3vs0_time_course.Rds",sep=""))

In [23]:
%%R
saveRDS(deleted_data, file=paste(OUT_DIR, "format_name_deleted_data3vs0_time_course.Rds", sep=""))